In [17]:
!python -m venv myenv

In [18]:
!pip install duckdb pandas polars numpy pyarrow

In [19]:
!aws s3 ls 

2023-12-16 17:32:52 amplify-photoapp-staging-173239-deployment
2021-12-29 13:41:30 aws-codestar-us-east-1-052552561229
2021-12-29 13:42:08 aws-codestar-us-east-1-052552561229-test-pipe
2022-01-08 14:33:48 aws-sam-cli-managed-default-samclisourcebucket-18d75byvsmhla
2021-10-31 04:32:21 aws-sam-cli-managed-default-samclisourcebucket-cda3h9le376y
2024-10-14 01:50:48 datagardeners-privacy
2021-12-02 16:12:57 ideastorm-bot-python
2021-05-01 02:39:09 ideastorm-bucket
2024-10-14 04:02:49 privacy.datagardeners.ai
2024-05-01 16:21:56 sagemaker-studio-052552561229-xvzbr6r7pvo
2021-12-02 16:13:50 sagemaker-studio-07ujk3wwqh9l
2024-05-01 16:21:58 sagemaker-us-east-1-052552561229
2022-11-06 01:10:36 sam-app-dev-pipelineartifactsbucket-vzxn3ld8taft
2022-11-06 01:10:12 sam-app-dev-pipelineartifactsloggingbucket-1ky5tskm1ghdt


In [20]:
import duckdb
import pandas as pd

In [21]:
con = duckdb.connect()
con.install_extension("aws")
con.load_extension("aws")

con.install_extension("s3")
con.load_extension("s3")

con.install_extension("httpfs")
con.load_extension("httpfs")

In [22]:
CRAWL_SET = "CC-MAIN-2024-42"
CC_URL_S3_BASE="s3://"
CC_URL_HTTP_BASE="https://data.commoncrawl.org/"
CC_INDEX_URL = f"{CC_URL_HTTP_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_BASE_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/"
CC_INDEX_URL

'https://data.commoncrawl.org/commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [23]:
CC_INDEX_S3_URL

's3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [24]:
print(f"{CC_INDEX_S3_BASE_URL}")

s3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/


In [25]:
# Configure S3 settings for DuckDB
con.execute("""
    CALL load_aws_credentials();
""")

In [26]:
con.execute("""
  CREATE SECRET secret3 (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN,
    CHAIN 'sso'
  );
            """)

In [27]:
# Read first few rows of Common Crawl index without downloading full dataset
try:
    query = f"""
        SELECT *
        FROM read_parquet('{CC_INDEX_S3_URL}')
        LIMIT 10
    """
    df = con.execute(query).fetchdf()
    print("First 5 rows of Common Crawl index:")
    print(df.head())
except Exception as e:
    print(f"Error reading Common Crawl index: {str(e)}")


First 5 rows of Common Crawl index:
                                         url_surtkey  \
0  com,blogspot,blogdocappacete)/2012/11/a-breve-...   
1  com,blogspot,blogdocappacete)/2012/11/burguesi...   
2  com,blogspot,blogdocappacete)/2012/11/deputado...   
3  com,blogspot,blogdocappacete)/2012/11/enquanto...   
4  com,blogspot,blogdocappacete)/2012/11/matanca-...   

                                                 url  \
0  https://blogdocappacete.blogspot.com/2012/11/a...   
1  https://blogdocappacete.blogspot.com/2012/11/b...   
2  https://blogdocappacete.blogspot.com/2012/11/d...   
3  https://blogdocappacete.blogspot.com/2012/11/e...   
4  https://blogdocappacete.blogspot.com/2012/11/m...   

                  url_host_name url_host_tld url_host_2nd_last_part  \
0  blogdocappacete.blogspot.com          com               blogspot   
1  blogdocappacete.blogspot.com          com               blogspot   
2  blogdocappacete.blogspot.com          com               blogspot   
3  blo

In [28]:
con.execute(f"""
  CREATE TABLE IF NOT EXISTS t1 as 
  SELECT *
    FROM read_parquet('{CC_INDEX_S3_URL}')
    LIMIT 100;
  DESCRIBE t1;
""").df()

,column_name,column_type,null,key,default,extra
0,url_surtkey,VARCHAR,YES,None,None,None
1,url,VARCHAR,YES,None,None,None
2,url_host_name,VARCHAR,YES,None,None,None
3,url_host_tld,VARCHAR,YES,None,None,None
4,url_host_2nd_last_part,VARCHAR,YES,None,None,None
5,url_host_3rd_last_part,VARCHAR,YES,None,None,None
6,url_host_4th_last_part,VARCHAR,YES,None,None,None
7,url_host_5th_last_part,VARCHAR,YES,None,None,None
8,url_host_registry_suffix,VARCHAR,YES,None,None,None
9,url_host_registered_domain,VARCHAR,YES,None,None,None


In [1]:
x = con.execute(f""" select warc_filename from t1 where url_host_name = 'blogdocappacete.blogspot.com' limit 10; """).df().iloc[0].iloc[0]
print(x)

NameError: name 'con' is not defined

In [41]:
!wget {CC_URL_HTTP_BASE}{x} -P data/

--2024-12-12 16:58:19--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-42/segments/1727944591543.93/warc/CC-MAIN-20241015204737-20241015234737-00610.warc.gz
Resolving data.commoncrawl.org (data.commoncrawl.org)... 3.167.163.111, 3.167.163.74, 3.167.163.83, ...
Connecting to data.commoncrawl.org (data.commoncrawl.org)|3.167.163.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948224248 (904M) [application/octet-stream]
Saving to: ‘data/CC-MAIN-20241015204737-20241015234737-00610.warc.gz’

CC-MAIN-20241015204 100%[===================>] 904.30M  20.3MB/s    in 42s     

2024-12-12 16:59:02 (21.5 MB/s) - ‘data/CC-MAIN-20241015204737-20241015234737-00610.warc.gz’ saved [948224248/948224248]



In [16]:
#!cut -d' ' -f3- ../cc/cdx-00000 > ../cc/cdx-00000.jsonl 

# IGNORE EVERYTHING BELOW THIS LINE

In [15]:
!duckdb -c "\
  create table meta as SELECT * FROM '../cc/cdx-00000.jsonl'; \
  create view status_vw as Select status, count(status) as cnt from meta group by status order by cnt desc; \
  select * from meta; \
  select * from status_vw; \
" {DB_FILE}

/bin/bash: line 1: duckdb: command not found


In [ ]:
!duckdb -c ".databases" {DB_FILE}
#!duckdb -c ".open hello.db"

In [ ]:
!duckdb -c " \
create table metastats as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta group by tstatus order by cstatus desc; \
select * from metastats; \
" {DB_FILE}

In [ ]:
!duckdb -c " \
create table metastats_norobotstxt as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta where url not like '%robots.txt' group by tstatus order by cstatus desc; \
select * from metastats_norobotstxt; \
" {DB_FILE}

In [ ]:
#!duckdb -c "drop table metastats" {DB_FILE}

In [ ]:
first_line = ""
with open("../cc/cdx-00000", "r") as file:
    first_line = file.readline()

print(first_line)

In [ ]:
!echo "{first_line}" | cut -d' ' -f3- 

In [ ]:
import json
s = '{"url": "http://1.0.0.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ", "length": "2706", "offset": "890", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz"}'
json.loads(s)